In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

# MPI_Allreduce

<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

Like `MPI_Reduce`, `MPI_Allreduce` collects values from each processor and performs an operation on the collected values.   The difference is that all-reduce sends the result of the operation to all processors, not just the root processor.  Here, we use `MPI_Allreduce` to compute a standard deviation. 

**Note.** This example is not the best way to compute the variance and standard deviation, since it requires two communication calls.  A better way is to use the one-pass [Welford's Online Algorithm](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance).

In this example, we use `MPI_Allreduce` to compute the variance and standard deviation of a set of random numbers.  The strategy is : 

* Rank 0 allocates an array of length N and initializes entries to uniformly distributed random numbers in [0,1]

* Rank 0 then sub-divides the array into `nprocs` segments and sends each segment to a rank. 

* Each rank computes the sum of the array entries it receives

* Then, calling `MPI_Allreduce`, each rank will get the total sum. Each rank then computes the global mean $\bar{x}$. 

* Rank 'j` computes the sum

\begin{equation*}
M_j = \sum_{i=1}^{N_{local}} (x_{jp + i} - \bar{x})^2
\end{equation*}


* A call to `MPI_Reduce` then collects all local variances to node 0.  The standard is then computed by summing $M$ from all processors.   The standard deviation 

\begin{equation*}
S = \sqrt{\frac{\sum_{j=1}^P M_j}{N-1}}
\end{equation*}


We seed the random number generator with a fixed value so that we can expect the mean to be the same, regardless of how many processors we run the program on.   The value of the standard deviation should be close to 

\begin{equation*}
S \approx \frac{1}{\sqrt{12}}
\end{equation*}


In [2]:
%%file allreduce_demo.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

double random_number()
{
  return (double) rand() / (double) RAND_MAX ;
}

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank,nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    /* Length of large array */
    int N = 8192;
    int Nlocal = N/nprocs;

    double *x;
    int tag = 0;
    double et_setup;
    if (rank == 0)
    {
        /* Seed random number generater */
        srand(1234);
        
        /* Allocate memory to create data */
        x = malloc(N*sizeof(double));
        
        /* Initialize array */
        for(int i = 0; i < N; i++)
        {
            x[i] = random_number();
        }
        
        /* Distribute subdomains of array to processors */
        for(int p = 1; p < nprocs; p++)
        {
            int dest = p;
            MPI_Send(&x[p*Nlocal],Nlocal,MPI_DOUBLE,dest,tag,MPI_COMM_WORLD);
        }
    }
    else
    {
        int source = 0;
        x = malloc(Nlocal*sizeof(double));
        MPI_Recv(x,Nlocal,MPI_DOUBLE,source,tag,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
    }

    double sum = 0;
    for(int i = 0; i < Nlocal; i++)
    {
        sum += x[i];
    }

    /* Each processor will get the total sum */
    int root = 0;
    double total_sum;
    MPI_Allreduce(&sum,&total_sum,1,MPI_DOUBLE,MPI_SUM,MPI_COMM_WORLD);    
    double xbar = total_sum/N;

    /* Get sum of square deviations from the mean. */
    double s2 = 0;
    for(int i = 0; i < Nlocal; i++)
    {
        s2 += (x[i] - xbar)*(x[i] - xbar);
    }
    
    /* Collect sum on rank 0. */
    MPI_Reduce(&s2,&total_sum,1,MPI_DOUBLE,MPI_SUM,root,MPI_COMM_WORLD);  
    
    if (rank == 0)
    {
        double std = sqrt(total_sum/(N-1));
        printf("N = %d\n",N);
        printf("STD            : %.16f\n",std);
        printf("STD (expected) : %.16f\n",1./sqrt(12.0));
    }

    free(x);

    MPI_Finalize();
}

Overwriting allreduce_demo.c


In [3]:
%%bash

rm -rf allreduce_demo

mpicc -o allreduce_demo allreduce_demo.c -lm

time mpirun -n 4 allreduce_demo

N = 8192
STD            : 0.2866859640361239
STD (expected) : 0.2886751345948129



real	0m0.134s
user	0m0.031s
sys	0m0.122s
